In [1]:
import psycopg
import pandas as pd

In [2]:
# Cargar el CSV en un dataframe
csv_path = 'Online Sales Data.csv'
df = pd.read_csv(csv_path)

In [3]:
# Transformación: Calcular el precio total si no está presente
if 'Total Revenue' not in df.columns:
    df['Total Revenue'] = df['Units Sold'] * df['Unit Price']

In [6]:
# Renombrar columnas para coincidir con las columnas de la tabla en PostgreSQL
df.rename(columns={
    'Transaction ID': 'TransactionID',
    'Date': 'Date',
    'Product Category': 'ProductCategory',
    'Product Name': 'ProductName',
    'Units Sold': 'UnitsSold',
    'Unit Price': 'UnitPrice',
    'Total Revenue': 'TotalRevenue',
    'Region': 'Region',
    'Payment Method': 'PaymentMethod'
}, inplace=True)

In [9]:
# Conexion con PostgreSQL
conn = psycopg.connect(
    host="localhost",
    dbname="dbtest",
    user="postgres",
    password="69420",
    port=5432
)
cursor = conn.cursor()


In [7]:
# Crear la tabla si no existe
create_table_query = '''
CREATE TABLE IF NOT EXISTS SalesTransactions (
    TransactionID INT PRIMARY KEY,
    Date DATE NOT NULL,
    ProductCategory VARCHAR(255) NOT NULL,
    ProductName VARCHAR(255) NOT NULL,
    UnitsSold INT NOT NULL,
    UnitPrice DECIMAL(10, 2) NOT NULL,
    TotalRevenue DECIMAL(10, 2) NOT NULL,
    Region VARCHAR(255) NOT NULL,
    PaymentMethod VARCHAR(50) NOT NULL
);
'''
cursor.execute(create_table_query)
conn.commit()


In [8]:
# Insertar los datos en la tabla
insert_query = '''
INSERT INTO SalesTransactions (TransactionID, Date, ProductCategory, ProductName, UnitsSold, UnitPrice, TotalRevenue, Region, PaymentMethod)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
ON CONFLICT (TransactionID) DO NOTHING;
'''

# Insertar filas en la base de datos
for index, row in df.iterrows():
    cursor.execute(insert_query, (
        row['TransactionID'],
        row['Date'],
        row['ProductCategory'],
        row['ProductName'],
        row['UnitsSold'],
        row['UnitPrice'],
        row['TotalRevenue'],
        row['Region'],
        row['PaymentMethod']
    ))

# Confirmar los cambios
conn.commit()

In [12]:
#Query parte 2 Tarea 1:

parte2_query1= '''
SELECT 
    ProductCategory,
    Region,
    SUM(TotalRevenue) AS TotalSales
FROM 
    SalesTransactions
GROUP BY 
    ProductCategory, 
    Region
ORDER BY 
    ProductCategory, 
    Region;
'''

cursor.execute(parte2_query1)
conn.commit()

# Obtener todos los resultados
results = cursor.fetchall()

# Imprimir los resultados
print("ProductCategory\tRegion\tTotalSales")
for row in results:
    print(f"{row[0]}\t{row[1]}\t{row[2]}")

ProductCategory	Region	TotalSales
Beauty Products	Europe	2621.90
Books	North America	1861.93
Clothing	Asia	8128.93
Electronics	North America	34982.41
Home Appliances	Europe	18646.16
Sports	Asia	14326.52


In [16]:
#  Cerrar la conexión
cursor.close()
conn.close()